## NBIC Workflow Provena Toy Example

This relies om some pre-registered components and is intended to show an example provenance enabled workflow by integrating with the provena APIs. 

The actual computation/validation has been stripped from the source notebook - but hopefully it will be clear where this can be reintroduced

### Provena workflow configuration setup

In [1]:
# This is a small helper class which provides a config object for validation and
# a loader function
import workflow_config

# This is a helper function for managing authentication with Provena
import mdsisclienttools.auth.TokenManager as ProvenaAuth

In [2]:
# Provena config
stage = "DEV"
kc_endpoint = "https://auth.dev.rrap-is.com/auth/realms/rrap"
registry_endpoint = "https://registry-api.dev.rrap-is.com"

In [3]:
# sets up auth connections - could potentially open browser window if not signed
# in recently - caches in .tokens.json - ensure this is included in gitignore
provena_auth = ProvenaAuth.DeviceFlowManager(
    stage=stage,
    keycloak_endpoint=kc_endpoint
)

# expose the get auth function which is used for provena methods 
get_auth = provena_auth.get_auth

No storage or object provided, using default location: .tokens.json.
Using storage type: FILE.
Using DEVICE auth flow.
Attempting to generate authorisation tokens.

Looking for existing tokens in local storage.

Validating found tokens

Found tokens valid, using.



In [4]:
# Start by loading the config from the specified path 

# NOTE this could change from run to run - this holds all information required to run this model
config_path = "configs/example_workflow.json"
config = workflow_config.load_config(path=config_path)
config.pprint()


{
  "inputs": {
    "hourly_temperature": "10378.1/1709248",
    "relative_humidity": "10378.1/1709249",
    "wind_speed": "10378.1/1709251",
    "daily_mc_adf": "10378.1/1709252"
  },
  "associations": {
    "person": "10378.1/1709236",
    "organisation": "10378.1/1709247"
  },
  "workflow_configuration": {
    "workflow_template": "10378.1/1709259"
  }
}


In [5]:
# let's validate the workflow config - this fetches ALL items referenced in the
# workflow json to ensure the items are valid 
valid = config.validate_entities(registry_endpoint=registry_endpoint, auth=get_auth)

if not valid:
    print("FAILED VALIDATION")
    raise Exception("Workflow config validation exception occurred. See output above.")


Validating registered Provena entities in config
Validating registered input datasets...
Validating registered associations...
Validating registered associations...


### Model run integration
Now that the validation of the workflow configuration (incl. registered entitites) is complete - we can move into the example of running the model against this configuration.